# chap4 

## 연습문제 4.1
아래에 나온 Option에 대한 함수들을 모두 구현하라. 각 함수를 구현할 떄 그 함수가 어떤 일을 하고 어떤 상황에서 쓰일 것인지 생각해 볼 것. 이 함수들 각각의 용도를 잠시 후에 보게 될 것이다. 다음은 이 연습문제를 위한 몇 가지 힌트이다.
- 패턴 부합을 사용해도 좋으나, map과 getOrElse를 제외한 모든 함수는 패턴 부합 없이도 구현할 수 있어야한다.
- map과 flatMap의 형식 서명은 구현을 결정하기에 충분해야 한다.
- getOrElse는 Option의 Some 안의 결과를 돌려준다. 단, Option이 None이면 주어진 기본 값을 돌려준다.
- orElse는 첫 Option이 정의되어 있으면 그것을 돌려주고 그렇지 않으면 둘째 Option을 돌려준다.
```scala
def map[B](f: A => B): Option[B]
def flatMap[B](f: A => Option[B]): Option[B]
def getOrElse[B >: A](default: => B) : B
def orElse[B >: A](ob: => Option[B]): Option[B]
def filter(f: A => Boolean): Option[A]
```

In [1]:
sealed trait Option[+A] {
  def map[B](f: A => B): Option[B] = this match {
    case Some(x) => Some(f(x))
    case None => None
  }
  
  def flatMap[B](f: A => Option[B]): Option[B] = 
    this.map(f).getOrElse(None)
  
  def flatMap_[B](f: A => Option[B]): Option[B] = this match {
    case Some(x) => f(x)
    case None => None
  }
    
  def getOrElse[B >: A](default: => B): B = this match {
    case Some(x) => x
    case None => default
  }
  def orElse[B >: A](ob: => Option[B]): Option[B] = 
    map(Some(_)).getOrElse(ob)
  
  def filter(f: A => Boolean): Option[A] = this match {
    case Some(x) => if(f(x)) Some(x) else None
    case None => None
  }
  def filter_(f: A => Boolean): Option[A] = 
    flatMap(a => if (f(a)) this else None)
}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

object Option {
  def none[A]: Option[A] = None
  def some[A](a: A): Option[A] = Some(a)
  def apply[A](a: A): Option[A] = Some(a)
}

defined trait Option
defined class Some
defined object None
defined object Option

In [2]:
def mean(xs: Seq[Double]): Option[Double] = 
  if (xs.isEmpty) None
  else Some(xs.sum / xs.length)

defined function mean

In [3]:
mean(Seq(1.0,2.0,3.0,4.0,5))

res2: Option[Double] = Some(3.0)

## 연습문제 4.2
variance 함수를 flatMap을 이용해서 구현하라. 순차열의 평균이 m이라 할 때, 분산(variance)은 순차열의 각 요소 x에 대한 math.pow(x - m, 2) 들의 평균이다.
```scala
def variance(xs: Seq[Double]): Option[Double]
```

In [4]:
def variance(xs: Seq[Double]): Option[Double] = 
  mean(xs).flatMap(m => mean(xs.map(x => math.pow(x - m, 2))))

defined function variance

In [5]:
variance(Seq(1,2,3,4,5.0))

res4: Option[Double] = Some(2.0)

In [6]:
def lift[A,B](f: A => B): Option[A] => Option[B] = _ map f

defined function lift

In [7]:
def Try[A](a: => A): Option[A] =
  try Some(a)
  catch { case e: Exception => None}

defined function Try

## 연습문제 4.3
두 Option 값을 이항 함수(binary function)를 이용해서 결합하는 일반적 함수 map2를 작성하라.
두 Option 값 중 하나라도 None이면 map2의 결과 역시 None이어야 한다. 서명은 다음과 같다.
```scala
def map2[A,B,C](a: Option[A], b: Option[B])(f: (A, B) => C): Option[C]
```

In [8]:
def map2[A,B,C](a: Option[A], b: Option[B])(f: (A, B) => C): Option[C] = (a, b) match {
  case (Some(x), Some(y)) => Some(f(x,y))
  case _ => None
}

def map2_[A,B,C](a: Option[A], b: Option[B])(f: (A, B) => C): Option[C] =
  a flatMap(aa => b map (bb => f(aa, bb)))

defined function map2
defined function map2_

In [9]:
def insuranceRateQuote(age: Int, numberOfSpeedingTickets: Int): Double = ???

def parseInsuranceRateQuote(age: String, numberOfSpeedingTickets: String): Option[Double] = {
  val optAge: Option[Int] = Try(age.toInt)
  val optTickets: Option[Int] = Try(numberOfSpeedingTickets.toInt)
  map2(optAge, optTickets)(insuranceRateQuote)
}

defined function insuranceRateQuote
defined function parseInsuranceRateQuote

## 연습문제 4.4
Option들의 목록을 받고, 그 목록에 있는 모든 Some 값으로 구성된 목록을 담은 Option을 돌려주는 함수 sequence를 작성하라. 원래의 목록에 None이 하나라도 있으면 함수의 결과도 None이어야 한다. 그렇지 않으면 원래의 목록에 있는 모든 값의 목록을 담은 Some을 돌려주어야 한다. 서명은 다음과 같다.
```scala
def sequence[A](a: List[Option[A]]): Option[List[A]]
```

In [10]:
def sequence[A](a: List[Option[A]]): Option[List[A]] = a match {
    case h :: t => h.flatMap(x => sequence(t).map(x :: _))
    case Nil => Option(Nil)
}

defined function sequence

In [11]:
sequence(List(Option(1), Option(2)))

res10: Option[List[Int]] = Some(List(1, 2))

In [12]:
def parseInts(a: List[String]): Option[List[Int]] =
    sequence(a map (i => Try(i.toInt)))

defined function parseInts

## 연습문제 4.5
traverse 함수를 구현하라. map과 sequence를 사용하면 간단하겠지만, 목록을 단 한 번만 훑는 좀 더 효율적인 구현을 시도해 볼 것. 더 나아가서, sequence를 이 traverse로 구현해 보라.
```scala
def traverse[A,B](a: List[A])(f: A => Option[B]): Option[List[B]]
```

In [13]:
def traverse[A,B](a: List[A])(f: A => Option[B]): Option[List[B]] = a match {
    case Nil => Some(Nil)
    case h :: t => map2(f(h), traverse(t)(f))(_ :: _)
}
def traverse_[A,B](a: List[A])(f: A => Option[B]): Option[List[B]] = 
    sequence(a map f)

def sequenceViaTraverse[A](a: List[Option[A]]): Option[List[A]] = 
    traverse(a)(x => x)

defined function traverse
defined function traverse_
defined function sequenceViaTraverse

## Either

In [14]:
sealed trait Either[+E, +A] {
  def map[B](f: A => B): Either[E, B] = this match {
    case Right(a) => Right(f(a))
    case Left(e) => Left(e)
  }
}
case class Left[+E](value: E) extends Either[E, Nothing]
case class Right[+A](value: A) extends Either[Nothing, A]

object Either {
  def left[E](e: E): Either[E, A] = Left(e)
  def right[A](a: A): Either[E, A] = Right(a)
  def apply[A](a: A): Either[E, A] = Right(a)
}

defined trait Either
defined class Left
defined class Right

In [19]:
def mean(xs: IndexedSeq[Double]): Either[String, Double] =
  if (xs.isEmpty) Left("mean of empty list!")
  else Right(xs.sum/xs.length)

defined function mean

In [20]:
def safeDiv(x: Int, y: Int): Either[Exception, Int] = 
  try Right(x / y)
  catch { case e: Exception => Left(e) }

defined function safeDiv

In [21]:
def Try[A](a: => A): Either[Exception, A] =
  try Right(a)
  catch { case e: Exception => Left(e) }

defined function Try

## 연습문제 4.6
Right 값에 대해 작용하는 버전의 map, flatMap, orElse, amp2, Either를 구현하라.
```scala
trait Either[+E, +A] {
  def map[B](f: A => B): Either[E, B]
  def flatMap[EE >: E, B](f: A => Either[EE, B]): Either[EE, B]
  def orElse[EE >: E, B >: A](b: => Either[EE, B]): Either[EE, B]
  def map2[EE >: E, B, C](b: Either[EE, B])(f: (A, B) => C): Either[EE, C]
}
```

In [18]:
sealed trait Either[+E, +A] {
  def map[B](f: A => B): Either[E, B] = this match {
    case Right(a) => Right(f(a))
    case Left(e) => Left(e)
  }
  def flatMap[EE >: E, B](f: A => Either[EE, B]): Either[EE, B] = this match {
    case Right(a) => f(a)
    case Left(e) => Left(e)
  }
  def orElse[EE >: E, B >: A](b: => Either[EE, B]): Either[EE, B] = this match {
    case Right(a) => Right(a)
    case Left(e) => b
  }
  def map2[EE >: E, B, C](b: Either[EE, B])(f: (A, B) => C): Either[EE, C] = 
    flatMap (aa => b map (f(aa, _)))
  
  def map2_[EE >: E, B, C](b: Either[EE, B])(f: (A, B) => C): Either[EE, C] = 
    for {
      a <- this
      b1 <- b
    } yield f(a, b1)
}
case class Left[+E](value: E) extends Either[E, Nothing]
case class Right[+A](value: A) extends Either[Nothing, A]

object Either {
  def left[E,A](e: E): Either[E,A] = Left(e)
  def right[E,A](a: A): Either[E,A] = Right(a)
  def apply[E,A](a: A): Either[E, A] = Right(a)
}

defined trait Either
defined class Left
defined class Right
defined object Either

In [22]:
def parseInseranceRateQuote(age: String, numberOfSpeedingTickets: String): Either[Exception, Double] =
  for {
    a <- Try(age.toInt)
    b <- Try(numberOfSpeedingTickets.toInt)
  } yield insuranceRateQuote(a, b)

defined function parseInseranceRateQuote

## 연습문제 4.7
Either에 대한 sequence와 traverse를 작성하라. 이 두 함수는 발생한 첫 오류를 돌려주어야 한다(오류가 발생했다면).
```scala
def sequence[E, A](es: List[Either[E, A]]): Either[E, List[A]]
def traverse[E, A, B](as: List[A])(f: A => Either[E, B]): Either[E, List[B]]
```

In [26]:
def sequence_[E, A](es: List[Either[E, A]]): Either[E, List[A]] = es match {
  case h :: t => h.flatMap(x => sequence(t).map(x :: _))
  case Nil => Right(Nil)
}
  
def traverse[E, A, B](as: List[A])(f: A => Either[E, B]): Either[E, List[B]] = as match {
  case h :: t => f(h).map2(traverse(t)(f))(_ :: _)
  case Nil => Right(Nil)
}

def sequence[E, A](es: List[Either[E, A]]): Either[E, List[A]] =
  traverse(es)(x => x)

defined function sequence_
defined function traverse
defined function sequence